In [ ]:
!pip install datasets
!pip install peft
# !pip install unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.0 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
  

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets, Dataset
import pandas as pd
import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score
# from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    MistralForSequenceClassification,
    PretrainedConfig,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
# from unsloth import FastLanguageModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_location = "/content/drive/MyDrive/685 Final Project/Datasets/medmcqa-prompts"

In [ ]:
# train_dataset = load_from_disk(f"{dataset_location}/train_prompts.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts.hf")

# train_dataset = load_from_disk(f"{dataset_location}/train_prompts_micro.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts_micro.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_micro.hf")

train_dataset = load_from_disk(f"{dataset_location}/train_prompts_mini.hf")
test_dataset = load_from_disk(f"{dataset_location}/test_prompts_mini.hf")
eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_mini.hf")

In [ ]:
set(train_dataset['cop'])

{0, 1, 2, 3}

In [ ]:
num_classes = len(set(train_dataset['cop']))
num_classes

4

In [ ]:
train_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot'],
    num_rows: 20000
})

In [ ]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot'],
    num_rows: 2000
})

In [ ]:
model_name = "/content/drive/MyDrive/685 Final Project/Models/unsloth_domain1"

# qunatization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)


model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels= num_classes,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at unsloth/mistral-7b-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

MistralForSequenceClassification(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()


In [ ]:
# lora config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj',
                      "gate_proj", "up_proj", "down_proj",],
    # target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='/content/drive/MyDrive/685 Final Project/Models/unsloth_domain1', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# model = SequenceClassificationWithSoftmax(model)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): MistralForSequenceClassification(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lor

In [ ]:
model.print_trainable_parameters()


trainable params: 41,959,424 || all params: 7,152,635,904 || trainable%: 0.586628825556951


In [ ]:
# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['prompt'])
    # tokenized_inputs['labels'] = examples['cop']
    tokenized_inputs['labels'] = examples['label_one_hot']
    # print(f"examples['cop']: {examples['cop']}")
    return tokenized_inputs

In [ ]:
tokenized_train = train_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_train = tokenized_train.with_format('torch')
tokenized_train[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': tensor(2),
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'prompt': '\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve 

In [ ]:
tokenized_eval = eval_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_eval = tokenized_eval.with_format('torch')
tokenized_eval

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})

In [ ]:
tokenized_train['labels']

tensor([[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])

In [ ]:
tokenized_train['prompt'][0]

'\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: </s>'

In [ ]:
model.forward(tokenized_train['input_ids'][0].unsqueeze(0),tokenized_train['attention_mask'][0].unsqueeze(0))


SequenceClassifierOutputWithPast(loss={'logits': tensor([[ 8.1191,  5.7866, -1.9454,  3.4773]], grad_fn=<ToCopyBackward0>), 'past_key_values': ((tensor([[[[-1.3471e-01, -3.9546e-02,  2.8171e-01,  ..., -1.4295e+00,
           -2.0667e+00, -2.2187e+00],
          [ 4.1105e+00, -3.1447e+00, -1.1018e+00,  ...,  1.9692e+00,
            3.8417e-01, -4.7688e-01],
          [ 2.3845e+00, -2.6463e+00,  2.8804e-01,  ...,  2.0324e+00,
            3.5898e-01, -4.2796e-01],
          ...,
          [-4.5376e+00, -2.2732e+00,  1.0609e+00,  ...,  1.8260e+00,
            3.6062e-01, -4.5504e-01],
          [-1.5682e+00,  5.5490e-01,  2.5836e+00,  ...,  1.9549e+00,
            3.4008e-01, -5.0602e-01],
          [ 7.0219e-02,  2.1487e-01,  2.7214e-01,  ..., -3.5951e-03,
            9.1053e-02,  2.1674e-01]],

         [[ 6.5623e-02,  7.2567e-02,  5.9086e-02,  ...,  3.2736e+00,
           -3.4291e+00,  1.6999e+00],
          [ 5.0552e-01, -1.1052e+00, -2.4723e-01,  ..., -5.1635e+00,
            5.1398e+

In [ ]:
# define custom batch preprocessor
def collate_fn(batch, tokenizer):
## Adds Padding from the tokenizer.
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    # print(batch)
    # d = {k: [print(k, dic) for dic in batch] for k in dict_keys}
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    # d['cop'] = torch.stack(d['cop'])

    return d

# define which metrics to compute for evaluation
def compute_metrics(p):
    predictions, labels = p

    predictions = np.argmax(predictions, axis=1)
    labels = np.argmax(labels, axis=1)

    accuracy = accuracy_score(labels, predictions)
    return {
        'accuracy': accuracy
    }

In [ ]:
tokenized_train[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': tensor(2),
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'prompt': '\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve 

In [ ]:
# collate_fn(tokenized_train[0:8], tokenizer)
g = functools.partial(collate_fn, tokenizer=tokenizer)
g(tokenized_train.select(range(8)))

{'input_ids': tensor([[    1, 28705,    13,  ...,     2,     2,     2],
         [    1, 28705,    13,  ...,     2,     2,     2],
         [    1, 28705,    13,  ...,     2,     2,     2],
         ...,
         [    1, 28705,    13,  ...,     2,     2,     2],
         [    1, 28705,    13,  ...,     2,     2,     2],
         [    1, 28705,    13,  ...,     2,     2,     2]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]])}

In [ ]:
type(tokenized_train[0:8])

dict

In [ ]:
# create custom trainer class to be able to pass label weights and calculate multiclass loss
class CustomTrainer(Trainer):

    def __init__(self, label_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # print(f"labels: {labels}")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # print(f"logits: {logits}")
        # compute custom loss
        loss = F.cross_entropy(logits, labels, weight=self.label_weights)
        # print(f"loss: {loss}")


        # probs = F.softmax(logits, dim=-1)
        # b_loss = F.binary_cross_entropy(probs, labels.to(torch.float32))
        # print(f"b_loss: {b_loss}")

        return (loss, outputs) if return_outputs else loss

In [ ]:
tokenized_train.select(range(8))

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8
})

In [ ]:
#### Train on single data point #####
training_args = TrainingArguments(
    output_dir = 'multilabel_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    logging_steps = 1
)

# train
trainer = CustomTrainer(
    model = model,
    args = training_args,
    # train_dataset = tokenized_train,
    train_dataset = tokenized_train.select(range(10)),
    # eval_dataset = tokenized_eval,
    eval_dataset = tokenized_eval.select(range(10)),
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    # label_weights = torch.tensor(label_weights, device=model.device)
)
# trainer.add_callback(accuracy_callback)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,6.601600,8.369576,0.200000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=5, training_loss=5.265813314914704, metrics={'train_runtime': 9.1305, 'train_samples_per_second': 1.095, 'train_steps_per_second': 0.548, 'total_flos': 51313589010432.0, 'train_loss': 5.265813314914704, 'epoch': 1.0})

In [ ]:
# define training args
training_args = TrainingArguments(
    output_dir = 'multilabel_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    logging_steps = 100
)

# train
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    # train_dataset = tokenized_train.select(range(128)),
    eval_dataset = tokenized_eval,
    # eval_dataset = tokenized_eval.select(range(128)),
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    # label_weights = torch.tensor(label_weights, device=model.device)
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.978300,1.100294,0.507500


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=625, training_loss=1.1701091857910155, metrics={'train_runtime': 10416.197, 'train_samples_per_second': 1.92, 'train_steps_per_second': 0.06, 'total_flos': 1.672961186117714e+17, 'train_loss': 1.1701091857910155, 'epoch': 1.0})

In [ ]:
saved_model_location = "/content/drive/MyDrive/685 Final Project/Models"

In [ ]:
####### model.save_pretrained(f"{saved_model_location}/Mistral_MedMCQ_Micro_dataset")
###### model.save_pretrained(f"{saved_model_location}/Mistral_MedMCQ_Mini_dataset")
# model.save_pretrained(f"{saved_model_location}/Mistral_MedMCQ_Mini_dataset_cross_entropy")
model.save_pretrained(f"{saved_model_location}/Domain_MedMCQ_Mini_dataset_cross_entropy")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
